In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [66]:
path='/content/drive/MyDrive/01 Data Science with Python ML deep learning/ML/Project 02 Insurance ML Regression/insurance.csv'

In [67]:
df = pd.read_csv(path)
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Step 1. Preprocessing


In [68]:
df['age'].unique()

array([19, 18, 28, 33, 32, 31, 46, 37, 60, 25, 62, 23, 56, 27, 52, 30, 34,
       59, 63, 55, 22, 26, 35, 24, 41, 38, 36, 21, 48, 40, 58, 53, 43, 64,
       20, 61, 44, 57, 29, 45, 54, 49, 47, 51, 42, 50, 39])

In [69]:
df['sex'].unique()

array(['female', 'male'], dtype=object)

In [70]:
df['children'].unique()

array([0, 1, 3, 2, 5, 4])

In [71]:
df['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [72]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

# Step 2. EDA

In [73]:
continous_feature = ['age', 'bmi','charges' ]
discrete_feature = ['sex','children','smoker', 'region']

In [74]:
df[continous_feature].describe()

,age,bmi,charges
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,13270.422265
std,14.049960,6.098187,12110.011237
min,18.000000,15.960000,1121.873900
25%,27.000000,26.296250,4740.287150
50%,39.000000,30.400000,9382.033000
75%,51.000000,34.693750,16639.912515
max,64.000000,53.130000,63770.428010


In [75]:
df[discrete_feature].corr()

<ipython-input-75-8d136ab9303c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[discrete_feature].corr()


,children
children,1.0


# Step 3 ---> Data Preprocessing

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [77]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [78]:
df.duplicated().sum()

1

In [79]:
df.drop_duplicates(inplace=True)

In [80]:
df.shape

(1337, 7)

In [81]:
#drop region code
df.drop(columns=['region'], inplace=True)

In [82]:
df['sex'].replace({'female':0, 'male':1}, inplace=True)
df['smoker'].replace({'no':0, 'yes':1},inplace=True)

In [83]:
X=df.drop('charges', axis=1)
y=df['charges']

In [84]:
X

,age,sex,bmi,children,smoker
0,19,0,27.900,0,1
1,18,1,33.770,1,0
2,28,1,33.000,3,0
3,33,1,22.705,0,0
4,32,1,28.880,0,0
...,...,...,...,...,...
1333,50,1,30.970,3,0
1334,18,0,31.920,0,0
1335,18,0,36.850,0,0
1336,21,0,25.800,0,0


In [85]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1337, dtype: float64

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=9)

# lasso regression
---

with default

In [87]:
#modelling
trn=[]
tst=[]
from sklearn.linear_model import Lasso
lasso_base=Lasso()
lasso_base.fit(X_train,y_train)

print('Intercept : ', lasso_base.intercept_)
print('Coeficient : ', lasso_base.coef_)
#prediction
train_pred = lasso_base.predict(X_train)
test_pred = lasso_base.predict(X_test)
#evaluation
print("Train R2", lasso_base.score(X_train,y_train))
print("Test R2", lasso_base.score(X_test,y_test))

Intercept :  -12149.348570420703
Coeficient :  [  264.55881378  -117.41603504   318.32086199   411.254903
 23982.65590602]
Train R2 0.7593597132035765
Test R2 0.7006279150592305


hyper parameter tunning for lasso

In [88]:
from sklearn.model_selection import GridSearchCV

#model
estimator = Lasso()

#parameter and values
param_grid = {'alpha':list(range(1,30))}

#identify the best value of the parameter within given values for the guven data
model_hp = GridSearchCV(estimator,param_grid,cv=5, scoring='r2')
model_hp.fit(X_train,y_train)

model_hp.best_params_



{'alpha': 29}

Result lasso model using best hyperparameter

# Redge Regression
-----

In [90]:
from sklearn.linear_model import Ridge

ridge_base=Lasso()
ridge_base.fit(X_train,y_train)

print('Intercept : ', ridge_base.intercept_)
print('Coeficient : ', ridge_base.coef_)
#prediction
train_pred = ridge_base.predict(X_train)
test_pred = ridge_base.predict(X_test)
#evaluation
print("Train R2", ridge_base.score(X_train,y_train))
print("Test R2", ridge_base.score(X_test,y_test))

Intercept :  -12149.348570420703
Coeficient :  [  264.55881378  -117.41603504   318.32086199   411.254903
 23982.65590602]
Train R2 0.7593597132035765
Test R2 0.7006279150592305


In [94]:
from sklearn.model_selection import GridSearchCV

#model
estimator = Ridge()

#parameter and values
param_grid = {'alpha':list(range(1,30))}

#identify the best value of the parameter within given values for the guven data
model_hp = GridSearchCV(estimator,param_grid,cv=5, scoring='r2')
model_hp.fit(X_train,y_train)

model_hp.best_params_


{'alpha': 1}

# Elasticnet
----


In [95]:
from sklearn.linear_model import ElasticNet
eltnet=ElasticNet()
eltnet.fit(X_train,y_train)

print('Intercept : ', eltnet.intercept_)
print('Coeficient : ', eltnet.coef_)
#prediction
train_pred = eltnet.predict(X_train)
test_pred = eltnet.predict(X_test)
#evaluation
print("Train R2", eltnet.score(X_train,y_train))
print("Test R2", eltnet.score(X_test,y_test))

Intercept :  -8509.368578163054
Coeficient :  [ 254.9619872   354.17313814  327.80908754  444.66863812 6012.57063085]
Train R2 0.40398031945307333
Test R2 0.3684271884092043


In [98]:
from sklearn.model_selection import GridSearchCV

#model
estimator = ElasticNet()

#parameter and values
param_grid = {'alpha':list(range(1,30)),'l1_ratio':[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}

#identify the best value of the parameter within given values for the guven data
model_hp = GridSearchCV(estimator,param_grid,cv=5, scoring='r2')
model_hp.fit(X_train,y_train)

model_hp.best_params_


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.122e+10, tolerance: 1.313e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.215e+10, tolerance: 1.326e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

{'alpha': 29, 'l1_ratio': 1}

In [ ]:
#modeling

enr_best=ElasticNet()